# 3. Inverse Kinematic

In [7]:
import os
import numpy as np
import opensim as osim

In [2]:
# root directory of the opensim project
root = '/home/romain/Downloads/opensim/'

# participant
participant = 'SylGaud'

# model file name
scaled_model_name = 'scaled-model-MKR.osim'

# conf inverse kinematic
ik_conf_name = 'conf_ik.xml'

In [3]:
scaled_model = {
    'name': scaled_model_name,
    'path': os.path.join(root, 'metadata', participant, scaled_model_name)
}

ik = {
    'trc_path': os.path.join(root, 'outputs', participant, '0_trc'),
    'mot_path': os.path.join(root, 'outputs', participant, '2_inverse_kinematic'),
    'conf': os.path.join(root, 'metadata', 'generic', ik_conf_name),
    'output': os.path.join(root, 'metadata', participant, ik_conf_name)
}

In [4]:
# 0. load and define model
my_model = osim.Model(scaled_model['path'])

# 1. InverseKinematicsTool
ik_tool = osim.InverseKinematicsTool(ik['conf'])  # initialize InverseKinematicsTool from setup file
ik_tool = osim.InverseKinematicsTool()
ik_tool.setModel(my_model)
print(f"run inverse kinematic... (saved at {ik['mot_path']})")

for ifile in os.listdir(ik['trc_path']):
    print(f'\t{ifile}')
    # 2. set name of input trc file and output motion in tool
    ik_tool.setName(ifile.replace('.trc', ''))
    ik_tool.setMarkerDataFileName(os.path.join(ik['trc_path'], ifile))
    ik_tool.setOutputMotionFileName(os.path.join(ik['mot_path'], ifile.replace('trc', 'mot')))
    
    ik_tool.setResultsDir(ik['mot_path'])
    
    # 3. use the trc file to get the start and end times
    marker_data = osim.MarkerData(os.path.join(ik['trc_path'], ifile))
    ik_tool.setStartTime(marker_data.getStartFrameTime())
    ik_tool.setEndTime(marker_data.getLastFrameTime()-1e-2)  # -1e-2 because remove last frame resolve some bug
    
    # 4. save the settings in a setup file
    ik_tool.printToXML(ik['output'])
    ik_tool.run()

run inverse kinematic... (saved at /home/romain/Downloads/opensim/outputs/SylGaud/2_inverse_kinematic)
	marcheRapide02.trc
	marcheRapide04.trc
	marcheRapide01.trc
	jump03.trc
	marcheAccel01.trc
	SylGaud0.trc
	marcheAccel04.trc
	marcheRapide03.trc
	marcheAccel05.trc
	marcheRapide05.trc
	marcheAccel03.trc
	jump05.trc
	marcheAccel02.trc
	jump04.trc
	jump01.trc
	jump02.trc


## Check inverse kinematic errors

In [71]:
def display_error(x, threshold):
    err = []
    if x[2] > 0.3:
        err.append('rms')
    if x[3] > 0.5:
        err.append('max')
    
    if err:
        print(f'time: {x[0]}\tRMS: {x[2]*10:.1f} cm\tmax: {x[3]*10:.1f} cm')

In [74]:
# 5. check errors (marker error < 2-4 cm and RMS < 2 cm)
print('Warning, the following frames contain errors above the opensim guidelines:\n')
for ifile in os.listdir(ik['mot_path']):
    if ifile.endswith('sto'):
        print(ifile)
        errors = np.loadtxt(os.path.join(ik['mot_path'], ifile), skiprows=7)
        np.apply_along_axis(display_error, axis=1, arr=errors)
        print('----------------\n')

Warning, the following frames contain errors above the opensim guidelines:

marcheRapide04_ik_marker_errors.sto
time: 0.28	RMS: 1.4 cm	max: 5.0 cm
time: 0.29	RMS: 1.4 cm	max: 5.1 cm
time: 0.3	RMS: 1.4 cm	max: 5.1 cm
time: 0.31	RMS: 1.4 cm	max: 5.2 cm
time: 0.32	RMS: 1.4 cm	max: 5.2 cm
time: 0.33	RMS: 1.4 cm	max: 5.3 cm
time: 0.34	RMS: 1.5 cm	max: 5.4 cm
time: 0.35	RMS: 1.5 cm	max: 5.5 cm
time: 0.36	RMS: 1.5 cm	max: 5.6 cm
time: 0.37	RMS: 1.6 cm	max: 5.7 cm
time: 0.38	RMS: 1.6 cm	max: 5.8 cm
time: 0.39	RMS: 1.6 cm	max: 6.0 cm
time: 0.4	RMS: 1.7 cm	max: 6.2 cm
time: 0.41	RMS: 1.7 cm	max: 6.4 cm
time: 0.42	RMS: 1.8 cm	max: 6.6 cm
time: 0.43	RMS: 2.4 cm	max: 8.1 cm
time: 0.44	RMS: 2.8 cm	max: 7.7 cm
time: 0.45	RMS: 2.8 cm	max: 8.0 cm
time: 0.46	RMS: 3.0 cm	max: 8.2 cm
time: 0.47	RMS: 3.7 cm	max: 10.2 cm
time: 0.48	RMS: 3.8 cm	max: 10.4 cm
time: 0.49	RMS: 2.9 cm	max: 9.8 cm
time: 0.5	RMS: 3.1 cm	max: 10.2 cm
time: 0.51	RMS: 3.7 cm	max: 10.8 cm
time: 0.52	RMS: 3.9 cm	max: 11.0 cm
time: 0.53	